<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 01: Feature Backfill for SOLANA bitcoin</span>


## 🗒️ The tasks of this script
1. Download historical prices for SOLANA and Bitcoin as CSV files
2. Update the path of the CSV files in this notebook to point to the ones that you downloaded
5. Create an account on www.hopsworks.ai and get your HOPSWORKS_API_KEY
6. Run notebook to upload the feature on a hopsworks feature storage



### <span style='color:#ff5f27'> 📝 Imports

In [1]:
test_mode = False

In [2]:
import pandas as pd
import hopsworks
from utils import *
import json
import os
import warnings
from dotenv import load_dotenv

warnings.filterwarnings("ignore")
pd.set_option('display.float_format', '{:.0f}'.format)


### IF YOU WANT TO WIPE OUT ALL OF YOUR FEATURES AND MODELS, run the cell below

In [3]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
# #os.environ["HOPSWORKS_API_KEY"] = os.getenv("HOPSWORKS_API_KEY")

# #proj = hopsworks.login()
#util.purge_project(proj)

### Connect to hopsworks and upload historical data

---

In [4]:
load_dotenv()
#os.environ["HOPSWORKS_API_KEY"] = os.getenv("HOPSWORKS_API_KEY")
project = hopsworks.login()

2025-01-04 22:00:49,519 INFO: Initializing external client
2025-01-04 22:00:49,519 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-04 22:00:51,192 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1160346


In [5]:
#secrets = secrets_api(project.name)
#try:
#    secrets.create_secret("CRYPTO_API_KEY", CRYPTO_API_KEY)
#except hopsworks.RestAPIError:
#    CRYPTO_API_KEY = secrets.get_secret("CRYPTO_API_KEY").value

### Add historical data to hopsworks feature storage

#### Add historical solana prices

In [6]:
hist_data_sol = pd.read_csv("data/historical_solana.csv")
hist_data_sol = hist_data_sol[["TIMESTAMP", 'OPEN', 'HIGH', 'LOW', 'CLOSE', "VOLUME", 'VOLUME_BUY', 'VOLUME_SELL']]
hist_data_sol.columns = hist_data_sol.columns.str.lower()
hist_data_sol.head()

,timestamp,open,high,low,close,volume,volume_buy,volume_sell
0,1623888000,40,41,38,39,6008,2886,3122
1,1623974400,39,39,35,37,13557,7925,5632
2,1624060800,37,37,35,35,16986,9258,7728
3,1624147200,35,36,31,35,38681,21053,17629
4,1624233600,35,35,26,27,41904,15381,26523


In [7]:
fs = project.get_feature_store() 
solana_fg = fs.get_or_create_feature_group(
    name='solana',
    description='Solana price',
    version=1,
    primary_key=["timestamp"])

if not test_mode:
    solana_fg.insert(hist_data_sol)  

# solana_fg.update_feature_description("date", "Date")
#solana_fg.update_feature_description("price", "The price of Solana")
# solana_fg.update_feature_description("open", "The opening price of Solana")
#solana_fg.update_feature_description("high", "The highest price of Solana")
#solana_fg.update_feature_description("low", "The lowest price of Solana")
#solana_fg.update_feature_description("vol", "Volume")
#solana_fg.update_feature_description("change", "Change in price")

Uploading Dataframe: 100.00% |██████████| Rows 1286/1286 | Elapsed Time: 00:01 | Remaining Time: 00:00


#### Add historical data for bitcoin

In [8]:
hist_data_btc = pd.read_csv("data/historical_bitcoin.csv")
hist_data_btc = hist_data_btc[["TIMESTAMP", 'OPEN', 'HIGH', 'LOW', 'CLOSE', "VOLUME", 'VOLUME_BUY', 'VOLUME_SELL']]
hist_data_btc.columns = hist_data_btc.columns.str.lower()
hist_data_btc.head()

,timestamp,open,high,low,close,volume,volume_buy,volume_sell
0,1622678400,37565,39471,37159,39197,5230,2452,2778
1,1622764800,39197,39250,35577,36848,4992,2199,2793
2,1622851200,36848,37936,34826,35535,5533,2475,3057
3,1622937600,35535,36480,35251,35789,3188,1668,1519
4,1623024000,35789,36796,33334,33588,6166,2555,3611


In [9]:
fs = project.get_feature_store() 
bitcoin_fg = fs.get_or_create_feature_group(
    name='bitcoin',
    description='Bitcoin price',
    version=1,
    primary_key=["timestamp"])

if not test_mode:
    bitcoin_fg.insert(hist_data_btc)

Uploading Dataframe: 100.00% |██████████| Rows 1300/1300 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: bitcoin_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1160346/jobs/named/bitcoin_1_offline_fg_materialization/executions


#### Add historical data for fear and greed index

In [10]:
import requests
import pandas as pd
import io


url = "https://api.alternative.me/fng/?"

# Set the parameters for the API request
params = {
    "limit": 0 # Number of days of data to retrieve
}

fng_response = trigger_request(url, params)
data = fng_response['data']

In [11]:
fng_df = pd.DataFrame(data)
fng_df.drop(columns=["time_until_update"], inplace=True)
fng_df['timestamp'] = fng_df['timestamp'].astype(float)
fng_df = fng_df.rename(columns={'value': 'fng_value', 'value_classification': 'fng_classification'}) 
fng_df

,fng_value,fng_classification,timestamp
0,73,Greed,1735948800
1,74,Greed,1735862400
2,70,Greed,1735776000
3,66,Greed,1735689600
4,64,Greed,1735603200
...,...,...,...
2521,11,Extreme Fear,1517788800
2522,24,Extreme Fear,1517702400
2523,40,Fear,1517616000
2524,15,Extreme Fear,1517529600


In [12]:
fs = project.get_feature_store() 
fng_fg = fs.get_or_create_feature_group(
    name='f_n_g_index',
    description='fear_and_greed_index',
    version=1,
    primary_key=["timestamp"])

if not test_mode:
    fng_fg.insert(fng_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1160346/fs/1151049/fg/1393533


Uploading Dataframe: 100.00% |██████████| Rows 2526/2526 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: f_n_g_index_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1160346/jobs/named/f_n_g_index_1_offline_fg_materialization/executions


#### Enter a description for each feature in the Feature Group

In [13]:
fng_fg.update_feature_description("timestamp", "Timestamp of the Fear and Greed Index")
fng_fg.update_feature_description("fng_value", "Fear and Greed Index value")
fng_fg.update_feature_description("fng_classification", "Fear and Greed Index classification")
